# Preprocessing

In questo script prepariamo in un unico dataset tutti i dati necessari per l'EDA e per la costruzione dei modelli.

### Librerie e funzioni

In [1]:
from pathlib  import Path

import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point
from shapely.ops import nearest_points

import matplotlib.pyplot as plt
from datetime import datetime

import numpy as np

In [2]:
def date(YY,MM,DD):
    """
    Converte in datetime.date
    """
    return datetime.strptime(f'{YY}-{MM}-{DD}', '%Y-%m-%d').date()

def time(hh,mm,ss):
    """
    Converte in datetime.time
    """
    return datetime.strptime(f'{hh}:{mm}:{ss}', '%H:%M:%S').time()

def remove_weather(data):
    """
    Rimuove dal dataset 'weather' la serie temporale di dati meteo
    specificando quale tipo di dati meteo voler eliminare.
    
    data = 'winds' per i dati sul vento;
    data = 'precipitition' per quelli sulle precipitazioni;
    data = 'temperatures' per quelli sulle temperature.
    """
    remove = []
    #creo stringa per rimuovere dati su 'data'
    for HH in np.linspace(0,23,24,dtype='int'):
        for mm in np.linspace(0,45,4,dtype='int'):
            if HH < 10:
                if mm == 0:
                    remove.append(f'{data}.{0}{HH}{mm}{0}')
                else:
                    remove.append(f'{data}.{0}{HH}{mm}')
            else:
                if mm == 0:
                    remove.append(f'{data}.{HH}{mm}{0}')
                else:
                    remove.append(f'{data}.{HH}{mm}')
    weather.drop(columns=remove, inplace = True)
    
def fasce_1h(dataset):
    """
    Suddivide in fasce da un'ora i tweet per comune e per data.
    
    es: la fasciax comprenderà i tweet tra le x:00 e x:59.
    """
    for i in range(24):
        if i<10:
            fascia = [time(f'0{i}', '00','00'), time(f'0{i}', '59', '59')]
            dataset[f'fascia{i}'] = (dataset.time <= fascia[1]) & (dataset.time >= fascia[0])
        else:
            fascia = [time(f'{i}', '00','00'), time(f'{i}', '59', '59')]
            dataset[f'fascia{i}'] = (dataset.time <= fascia[1]) & (dataset.time >= fascia[0])
            
def nearest_station():
    """
    Crea in twitter la colonna nearest_station con i comuni in cui c'è una colonnina 
    più vicina al luogo in cui è stato fatto il tweet.
    """
    #geodataframe contenente le posizioni delle 36 colonnine e i relativi comuni
    posizione_colonnine = weather[['geometry', 'name']].drop_duplicates()
    #inserisco in twitter la colonna in cui andrò a inserire per ogni tweet il nome del comune con colonnina più vicino
    twitter.insert(3, 'nearest_station', None)
    multipoint = posizione_colonnine.geometry.unary_union
    #trovo posizione della colonnina più vicina per ogni tweet
    for index, row in twitter.iterrows():
        point = row.geometry
        queried_geom, nearest_geom = nearest_points(point, multipoint)
        twitter.loc[index, 'nearest_station'] = nearest_geom
    #sostituisco la posizione della colonnina con il nome del comune in cui si trova
    for i in range(twitter.shape[0]):
        index = posizione_colonnine.index[posizione_colonnine.geometry==twitter.nearest_station.loc[i]]
        twitter.loc[i, 'nearest_station'] = posizione_colonnine.name.loc[index.values[0]]

def insert_zero_tweet(twitter): 
    """
    Aggiunge a twitter le righe dei comuni con 0 tweet nelle corrispettive date.
    """
    #resetto indici
    twitter.reset_index(inplace=True)
    #definisco la lista delle date
    date_vector = twitter.groupby('date').count().index
    #ciclo per determinare le date con relativi comuni in cui non ci sono stati tweet
    colonne_mancanti=[]
    for d in date_vector:
        for n in name:
                if n in np.array(twitter[twitter['date'] == d]['name'].reset_index(drop=True)):
                    pass
                else:
                    colonne_mancanti += [[d, n]]
    #creo dataframe dei tweet mancanti e lo concateno a twitter
    mancante = pd.DataFrame(data = colonne_mancanti, columns=['date', 'name'])
    for c in twitter.columns[2:]:
        mancante[c] = np.zeros(mancante.shape[0])

    twitter = pd.concat([twitter,mancante])
    #rirpistino gli indici come in partenza
    twitter.set_index(['date', 'name'],inplace=True)
    twitter.sort_index(inplace=True)
    return twitter

### Importo dati

Utilizziamo i seguenti dataset.
* **trentino-grid.geojson**: la mappa del Trentito, trattabile importando il dataset come un geodataframe;  
* **meteotrentino-weather-station-data.json**: dati meteo del Trentino relativi a 36 colonnine meteo poste nella regione. I dati meteo sono vento (forza e direzione), precipitazioni (mm) e temperatura (°C) misurate nei giorni nell'intervallo [01/11/2013, 31/12/2013] ogni 15 minuti;  
* **social-pulse-trentino.geojson**: dati relativi all'attività twitter, i cui principali sono coordinate della posizione, data e ora della pubblicazione del tweet.

In [3]:
data_path = Path('./data/raw')

files = {'grid':'trentino-grid.geojson',
        'weather':'meteotrentino-weather-station-data.json',
        'twitter':'social-pulse-trentino.geojson'}

grid = gpd.read_file(data_path / files['grid'])
with open(data_path / files['weather']) as f:
    weather_json = json.load(f)
weather = gpd.GeoDataFrame(weather_json['features'])
with open(data_path / files['twitter']) as f:
    twitter_json = json.load(f)
twitter = gpd.GeoDataFrame(twitter_json['features'])

L'idea per il preprocessing è quella di fare un merge tra i dataset dei dati meteo e dell'attività twitter (che chiamiamo weather e twitter rispettivamente).  
Il dataset grid serve principalmente per plottare la mappa del Trentino.

### Preparo twitter e weather per il merge

#### Twitter

Inserisco colonna geometry per i plot, converto created in datetime type e separo la data dall'ora.  
Rimuovo inoltre le features superflue per l'analisi.

In [4]:
twitter['geometry'] = twitter['geomPoint.geom'].apply(lambda x:Point(x['coordinates'][0], x['coordinates'][1]))
twitter.drop(columns=['geomPoint.geom'],inplace=True)
twitter['created'] = pd.to_datetime(twitter['created'])
twitter['date'] = pd.to_datetime(twitter['created']).dt.date
twitter['time'] = pd.to_datetime(twitter['created']).dt.time

In [5]:
twitter.drop(columns = ['created', 'timestamp', 'user', 'municipality.acheneID', 'language', 'entities'], inplace=True)
twitter['name'] = twitter['municipality.name']
twitter.drop(columns=['municipality.name'], inplace=True)

#### Weather

Inserisco colonna geometry per i plot e converto la colonna 'date' nel formato datetime.date.

In [6]:
weather['geometry'] = weather['geomPoint.geom'].apply(lambda x:Point(x['coordinates'][0], x['coordinates'][1]))
weather.drop(columns=['geomPoint.geom'],inplace=True)
weather['date'] = pd.to_datetime(weather['date']).dt.date

Rimuovo i dati sul vento in quanto non sufficienti per i nostri scopi dato che mancano circa il 50% dei dati e quelli che ci sono non sono completi.  
Nello specifico ci sono colonnine che non misurano i dati sul vento, e quelle che lo fanno non lo fanno ogni 15 minuti ma sono presenti dei "vuoti" non ordinati nel tempo.  
Rimuovo inoltre features non significative per l'analisi.

In [7]:
weather.drop(columns=['timestamp', 'minWind', 'maxWind', 'minTemperature', 'maxTemperature'], inplace=True)
remove_weather('winds')

Inserisco nel dataset una colonna con il nome dei comuni associati ad ogni colonnina (nel dataset è presente infatti la colonna con i codici identificativi di ciascuna colonnina (la colonna 'station') che sono associabili ad un comune tramite la seguente pagina: https://www.meteotrentino.it/index.html#!/content?menuItemDesktop=111).   
Sostituisco quindi la colonna dei codici 'station' con quella dei comuni associati che chiamo 'name'.


In [8]:
station = weather.groupby('station').mean().index
name = ['Lavarone', 'Mezzana', "Malè", 'Moena', 'Zambana', 'Cembra', 'Trento(sud-est)', 'Trento(nord-ovest)', "Sant'Orsola Terme", 'Aldeno', 'Rovereto', 'Pinzolo', 'Tione', 'Montagne', 'Santa Massenza', 'Folgaria', 'Romeno', 'Passo Tonale', 'Cavalese', 'Dro', 'Molveno', 'Predazzo', 'Telve', 'Cles', 'Arco', 'Bezzecca', 'Ala', 'Mezzolombardo', 'Pergine Valsugana', 'San Lorenzo in Banale', 'Tonadico', 'Pieve di Bono', 'Capriana', 'Canazei', 'San Martino di Castrozza', 'Castello Tesino']
data = {'station':station, 'name':name}
id_name = pd.DataFrame(data = data)
weather = weather.merge(id_name)
weather.drop(columns=['station'], inplace=True)

#### Twitter (again)

Modifico ora il dataset 'twitter' come segue.
Dato che le colonnine sono 36 (una per comune a parte a Trento dove ce ne sono due, Trento(sud-est) e Trento(nord-ovest)) e i tweet sono fatti in più di 36 comuni, associo ogni tweet al comune contenente una colonnina più vicino. Questo per associare dei dati meteo a ciascun tweet. Chiamo questa colonna 'name'.

In [9]:
nearest_station()
twitter.drop(columns=['name'], inplace=True)
twitter.rename(columns = {'nearest_station':'name'}, inplace = True)

Ora ho associato ad ogni tweet il nome del comune contenente una colonnina meteo più vicino alla posizione da cui è stato fatto il tweet.  
Raggruppo ora il numero di tweet fatti per ognuno di questi comuni in fasce orarie da un'ora ciascuna per ogni data.  
Chiamo queste fasce 'fasciax' con x che va da 0 a 23. La fascia 'fascia3' ad esempio comprenderà i tweet fatti in un determinato comune (con colonnina) in un determinato giorno dall'ora 3:00 all'ora 3:59.

In [10]:
#Creo colonne fasciaHH con True se il dato cade in quella fasce e False altrimenti
fasce_1h(twitter)
twitter.drop(columns=['time'], inplace=True)
#ottengo un dataset con multindice data-luogo(comune con colonnina più vicino) e features numero di tweet per fascia oraria
twitter = twitter.groupby(['date', 'name']).sum()
twitter.head()

fascia0  fascia1  fascia2  fascia3  fascia4  \
date       name                                                           
2013-11-01 Aldeno                 0        0        0        0        0   
           Canazei                0        1        0        0        0   
           Castello Tesino        0        0        0        0        0   
           Cavalese               1        0        0        0        0   
           Cembra                 0        0        0        0        0   

                            fascia5  fascia6  fascia7  fascia8  fascia9  ...  \
date       name                                                          ...   
2013-11-01 Aldeno                 0        0        0        0        1  ...   
           Canazei                0        0        0        0        0  ...   
           Castello Tesino        0        0        0        0        0  ...   
           Cavalese               0        0        0        2        0  ...   
           Cembra                 0        0        0        0        1  ...   

                            fascia14  fascia15  fascia16  fascia17  fascia18  \
date       name                                                                
2013-11-01 Aldeno                  0         1         0         0         0   
           Canazei                 0         0         0         0         0   
           Castello Tesino         0         0         0         0         1   
           Cavalese                8         1         0         2         1   
           Cembra                  1         1         1         0         0   

                            fascia19  fascia20  fascia21  fascia22  fascia23  
date       name                                                               
2013-11-01 Aldeno                  0         0         0         1         0  
           Canazei                 0         2         1         1         0  
           Castello Tesino         2         0         0         0         0  
           Cavalese                0         0         0         2         1  
           Cembra                  0         2         0         0         0  

[5 rows x 24 columns]

**In twitter, per ogni data, non in tutti i comuni ci sono stati dei tweet**;
aggiungo quindi le righe dei comuni con 0 tweet nelle corrispettive date.

In [11]:
twitter = insert_zero_tweet(twitter)
twitter.head()

fascia0  fascia1  fascia2  fascia3  fascia4  fascia5  \
date       name                                                             
2013-11-01 Ala           0.0      0.0      0.0      0.0      0.0      0.0   
           Aldeno        0.0      0.0      0.0      0.0      0.0      0.0   
           Arco          0.0      0.0      0.0      0.0      0.0      0.0   
           Bezzecca      0.0      0.0      0.0      0.0      0.0      0.0   
           Canazei       0.0      1.0      0.0      0.0      0.0      0.0   

                     fascia6  fascia7  fascia8  fascia9  ...  fascia14  \
date       name                                          ...             
2013-11-01 Ala           0.0      0.0      0.0      0.0  ...       0.0   
           Aldeno        0.0      0.0      0.0      1.0  ...       0.0   
           Arco          0.0      0.0      0.0      0.0  ...       0.0   
           Bezzecca      0.0      0.0      0.0      0.0  ...       0.0   
           Canazei       0.0      0.0      0.0      0.0  ...       0.0   

                     fascia15  fascia16  fascia17  fascia18  fascia19  \
date       name                                                         
2013-11-01 Ala            0.0       0.0       0.0       0.0       0.0   
           Aldeno         1.0       0.0       0.0       0.0       0.0   
           Arco           0.0       0.0       0.0       0.0       0.0   
           Bezzecca       0.0       0.0       0.0       0.0       0.0   
           Canazei        0.0       0.0       0.0       0.0       0.0   

                     fascia20  fascia21  fascia22  fascia23  
date       name                                              
2013-11-01 Ala            0.0       0.0       0.0       0.0  
           Aldeno         0.0       0.0       1.0       0.0  
           Arco           0.0       0.0       0.0       0.0  
           Bezzecca       0.0       0.0       0.0       0.0  
           Canazei        2.0       1.0       1.0       0.0  

[5 rows x 24 columns]

#### Unisco i dati

Faccio ora il merge dei 2 dataset sugli indici 'date' e 'name' per avere come risultato un dataset che associa ad ogni data e ad ogni comune il numero di tweet per fascia oraria con i relativi dati meteo (temperatura e precipitazioni).

In [12]:
#imposto date e name come indici in weather per il merge
weather.set_index(['date', 'name'], inplace=True)
weather = weather.sort_index()
weather.drop(columns=['geometry', 'precipitation'], inplace=True)

In [13]:
twitter_weather = pd.merge(twitter,weather,left_index=True, right_index=True)
#resetto indici per comodità
twitter_weather.reset_index(inplace=True)
twitter_weather.head()

,date,name,fascia0,fascia1,fascia2,fascia3,fascia4,fascia5,fascia6,fascia7,...,precipitations.2130,precipitations.2145,precipitations.2200,precipitations.2215,precipitations.2230,precipitations.2245,precipitations.2300,precipitations.2315,precipitations.2330,precipitations.2345
0,2013-11-01,Ala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-11-01,Aldeno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-11-01,Arco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-11-01,Bezzecca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-11-01,Canazei,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Verifico che nel merge non abbia perso dei dati sull'attività twitter.

In [14]:
tot_twitter_merge = twitter_weather[[f'fascia{i}' for i in range(0,24)]].sum().sum()
tot_twitter = twitter[[f'fascia{i}' for i in range(0,24)]].sum().sum()
diff_tweet = tot_twitter-tot_twitter_merge
print(f'Ho perso {diff_tweet} tweet nel merge, che è lo {round(diff_tweet*100/tot_twitter,2)}% dei tweet totali.')

Ho perso 5.0 tweet nel merge, che è lo 0.02% dei tweet totali.


In [15]:
twitter.reset_index(inplace=True)
twitter[twitter['name'] == 'Mezzolombardo'].groupby('date').sum().sum(axis=1).iloc[8]

5.0

Ho perso nel merge circa lo 0.02% dei dati in quanto manca il 09/11/2013 (che è un sabato) nei dati meteo a Mezzolombardo, in cui sono stati fatti 5 tweet. Dato che mancano interamente i dati meteo e la frazione dell'attività twitter è trascurabile, sostituire i dati mancanti non ha importanza ai fini dell'analisi.

### Feature weekend/not weekend

Non abbiamo sufficienti dati per studiare una dipendenza del target dalla data specifica (servibbero anni di dati, per esempio, per prevedere un'attività twitter più elevata nel giorno di Capodanno o in altre festività) ma abbiamo un sufficiente numero di settimane (circa 8) per studiare un'eventuale dipendenza dalla feature categorica weekend/not weekend (1 weekend, 0 not weekend).  
Inseriamo quindi la colonna 'weekend' nel dataset.

Inserisco la colonna 'date_num' che associa, in ordine crescente, ad ogni data un numero da 1 a 61.

In [16]:
a = []
for i in range(1,62):
    if i != 9:
        a += [i for j in range(36)]
    else:
        a += [i for j in range(35)] #in questo giorno non ho dati sul comune di Mezzolombardo

twitter_weather['date_num'] = a

Il primo giorno del dataset è venerdì 2013-11-01. Categorizzo quindi la feature 'date_num' come detto in precedenza.

In [17]:
weekend = []
for i in a:
    if ((i-2)%7 == 0) | ((i-3)%7 == 0):
        weekend += [1]
    else:
        weekend += [0]

twitter_weather['weekend'] = weekend

In [18]:
twitter_weather.head()

,date,name,fascia0,fascia1,fascia2,fascia3,fascia4,fascia5,fascia6,fascia7,...,precipitations.2200,precipitations.2215,precipitations.2230,precipitations.2245,precipitations.2300,precipitations.2315,precipitations.2330,precipitations.2345,date_num,weekend
0,2013-11-01,Ala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,2013-11-01,Aldeno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,2013-11-01,Arco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,2013-11-01,Bezzecca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,2013-11-01,Canazei,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


### NaN

Nel dataset sono presenti dei valori NaN, e sono esclusivamente nei dati meteo. Probabilmente in corrispondenza di questi valori le colonnine meteo non li hanno registrati. 

In [19]:
twitter_weather.isna().sum().sum()

234

In [20]:
#nei dati twitter non ho NaN, infatti:
twitter_weather[[f'fascia{i}' for i in range(24)]].isna().sum().sum()

0

Sostituisco questi NaN con 0. Questo non influenza significativemente il training del modello in quanto 234 valori sono una percentuale minima dei dati totali.

In [21]:
twitter_weather.fillna(0, inplace=True)

### Salvataggio dataset

Salvo il dataset nel formato .csv nella cartella opportuna con il nome di 'twitter_weather.csv'.

In [22]:
save_path = Path('./data/processed')
twitter_weather.to_csv(save_path/'twitter_weather.csv', index = False)